In [1]:
from huggingface_hub import snapshot_download
model_base_path = "./models/Qwen_1_5B_base" 
snapshot_download(repo_id="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", local_dir=model_base_path)


/Users/freedomkwokmacbookpro/miniforge3/envs/openmanus-rl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 12692.92it/s]


'/Users/freedomkwokmacbookpro/Github/Kaggle/OpenRL/models/Qwen_1_5B_base'

In [3]:
from openmanus_rl.agentgym.agentenv.agentenv.prompts.gmail_calander import prompt as gmail_prompt, owner_tag, owner_email, owner_name, owner_perfernce, num_of_sentences, today
from openmanus_rl.agentgym.agentenv.agentenv.controller.task import ConversationMessage
from datetime import datetime

conversation_start: list[dict[str, str]] = [
            {
                "role": "user",
                "content": gmail_prompt
                    .replace(owner_tag, "owners")
                    .replace(owner_email, "freedomkwok2010@gmail.com")
                    .replace(owner_name, "Richard Guo")
                    .replace(today, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
                    .replace(owner_perfernce, "I try to schedule meeting at the morning")
            },
            {
                "role": "user",
                "content": "What is my avaliability tomorrow"
            }
        ]


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_trained = "/data/models/QWEN1_5B_MOD1"

tokenizer = AutoTokenizer.from_pretrained(model_trained, trust_remote_code=True)
prompt_text = tokenizer.apply_chat_template(conversation_start, tokenize=False, add_generation_prompt=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = tokenizer(prompt_text, return_tensors="pt").to(device)

model = AutoModelForCausalLM.from_pretrained(model_trained, trust_remote_code=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 14.95it/s]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qw

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=4300, temperature=0.6)

# Decode and print result
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜User｜>
You are an autonomous intelligent agent for managing Gmail calendars. There are two types of users who interact with you:the owner (who controls the calendar) and others (who do not). When the user is the owner, use list_events to find events. For others, use check_availability or find_free_time to find available time slots in the owner's calendar. 
Current User:owners
Owner Email:freedomkwok2010@gmail.com
Owner Name:Richard Guo
Today_Date:2025-07-09 12:43:22
Owner preference:I try to schedule meeting at the morning

Privacy:fields in private_field (creator, visibility, attendees, hangoutLink, summary, description, location) are omitted from outputs for owner.
When others schedule with the owner, please follow owner's preferences as closely as possible
We detail name, description, input(parameters) and output(returns) of each action as follows:

Actions:
Name:list_calendars
Description:List all available calendars in the user's Gmail account.
Parameters:
- None
Returns:
- A li

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

def print_trainable_params(model, note=""):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"--- {note} ---")
    print(f"Total parameters: {total:,}")
    print(f"Trainable parameters: {trainable:,}")
    print(f"Frozen parameters: {total - trainable:,}")
    print()

def count_named(model, pred):
    return sum(p.numel() for n, p in model.named_parameters() if pred(n, p))

# Paths
base_model_path = "/data/models/Qwen_1_5B"
adapter_path = "/data/models/SFT_0718_0442_55"
output_dir = "/data/models/QWEN1_5B_MOD1"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

# Base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,
    device_map="auto"
)

# IMPORTANT: freeze base before adding adapter
base_model.requires_grad_(False)
base_model.eval()

print_trainable_params(base_model, note="Before attaching adapter (base frozen)")

print("Loading adapter K0...")
peft_model = PeftModel.from_pretrained(base_model, adapter_path, adapter_name="K0")
peft_model.set_adapter("K0")

# Inspect adapter-specific trainables (e.g., lora layers)
lora_trainable = count_named(
    peft_model,
    lambda n, p: p.requires_grad and ("lora_" in n or "lora" in n or "adapter" in n)
)
print(f"Adapter-trainable parameter count (approx, name-filtered): {lora_trainable:,}")
print_trainable_params(peft_model, note="With adapter loaded (should show only adapter trainable)")

# Merge
print("Merging adapter K0...")
peft_model.merge_adapter()
merged_model = peft_model.merge_and_unload()

print_trainable_params(merged_model, note="After merging and unloading (all frozen)")

# Optional: verify weight changes (norm diff) on a few layers
with torch.no_grad():
    # Example layer names, adjust to your model’s naming
    for layer_name in ["model.embed_tokens.weight", "lm_head.weight"]:
        if hasattr(merged_model, "get_parameter"):
            try:
                w = merged_model.get_parameter(layer_name)
                print(f"{layer_name}: shape={tuple(w.shape)}, dtype={w.dtype}")
            except Exception:
                pass

# Save
merged_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Merged model saved to: {output_dir}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch, os

base_id = "/workspace/Qwen_1_5B"
A_path  = "/workspace/output/Qwen_1_5B/SFT_0815_0701_70"
B_path  = "/workspace/output/Qwen_1_5B/SFT_0815_0739_65"
C_path  = "/workspace/output/Qwen_1_5B/SFT_0815_0817_60"
output_dir = "/data/models/QWEN1_5B_0815_ABC"

tok = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)

# load base (fp16 is fine). device_map="auto" is OK for merge, but we’ll move to CPU before save.
base = AutoModelForCausalLM.from_pretrained(
    base_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

# load 3 adapters
model = PeftModel.from_pretrained(base, A_path, adapter_name="A")
model.load_adapter(B_path, adapter_name="B")
model.load_adapter(C_path, adapter_name="C")

# blend them into a new logical adapter
model.add_weighted_adapter(
    adapters=["A", "B", "C"],
    weights=[0.6, 0.2, 0.2],
    adapter_name="ABC_blend",
)
model.set_adapter("ABC_blend")

# MERGE: returns the plain HF model (no adapters)
merged = model.merge_and_unload()      # <-- capture the return!

# (optional) sanity: ensure it’s not a PeftModel anymore
print("Type after merge:", type(merged))

# consolidate to CPU so save_pretrained writes a proper set of files
merged = merged.to("cpu")

# write safetensors; bump shard size if you want a single file
merged.save_pretrained(
    output_dir,
    safe_serialization=True,           # <-- write .safetensors
    max_shard_size="10GB",             # optional: try to avoid multiple shards
)
tok.save_pretrained(output_dir)
print("Saved to:", output_dir)

In [ ]:
python3 -c "import torch; torch.cuda.empty_cache(); torch.cuda.ipc_collect(); print('✅ CUDA memory cache cleared.')"

In [ ]:
python3 -m vllm.entrypoints.openai.api_server \
  --model /workspace/qwen1_5B_K3_BF16 \
  --dtype bfloat16 \
  --port 38000 \
  --max-model-len 10000 \
  --gpu-memory-utilization 0.9